In [2]:
import os
from glob import glob
from tqdm import tqdm
os.chdir("..")
# --- CONFIGURACIÓN ---
LABELS_DIR = 'data/raw_originals/labels'
AUGMENT_FACTOR = 4

def analyze_flower_density():
    if not os.path.exists(LABELS_DIR):
        print(f"Error: No existe {LABELS_DIR}")
        return

    txt_files = glob(os.path.join(LABELS_DIR, '*.txt'))
    
    # Estructura para guardar datos: [ratio_flores, num_flores, num_arandanos]
    flower_images_data = []
    
    total_arandanos_global = 0
    total_flores_global = 0

    print(f"🧐 Escaneando {len(txt_files)} archivos...")

    for txt in tqdm(txt_files):
        with open(txt, 'r') as f:
            lines = [line.strip() for line in f.readlines() if line.strip()]
            
        n_arandanos = 0
        n_flores = 0
        
        for line in lines:
            cls_id = int(line.split()[0])
            if cls_id == 0:
                n_arandanos += 1
            elif cls_id == 1:
                n_flores += 1
        
        total_arandanos_global += n_arandanos
        total_flores_global += n_flores
        
        # Solo nos interesan imágenes que tengan al menos 1 flor
        if n_flores > 0:
            total_objs = n_arandanos + n_flores
            ratio = n_flores / total_objs  # 1.0 = Solo Flores, 0.1 = Pocas flores
            flower_images_data.append({
                'file': os.path.basename(txt),
                'ratio': ratio,
                'flores': n_flores,
                'arandanos': n_arandanos
            })

    # --- SIMULACIÓN DE ESCENARIOS ---
    # Vamos a probar 3 umbrales para ver cuál te conviene
    thresholds = [0.8, 0.5, 0.3] # 80%, 50%, 30% de flores en la imagen
    
    print("\n" + "="*60)
    print(f"ANÁLISIS DE DENSIDAD DE FLORES")
    print(f"   (Global actual: {total_flores_global} Flores vs {total_arandanos_global} Arándanos)")
    print("="*60)
    
    for thresh in thresholds:
        # Filtrar imágenes que superan el umbral
        candidates = [img for img in flower_images_data if img['ratio'] >= thresh]
        
        count_imgs = len(candidates)
        added_flores = sum(img['flores'] for img in candidates) * AUGMENT_FACTOR
        added_arandanos = sum(img['arandanos'] for img in candidates) * AUGMENT_FACTOR
        
        print(f"\n🔹 ESCENARIO: Solo aumentar imágenes con > {int(thresh*100)}% de Flores")
        print(f"   - Imágenes que cumplen: {count_imgs} (de {len(flower_images_data)} con flores)")
        
        if count_imgs == 0:
            print(f"Ninguna imagen cumple este criterio. Baja el umbral.")
            continue

        # Proyección
        final_flores = total_flores_global + added_flores
        final_arandanos = total_arandanos_global + added_arandanos
        
        ratio_impact = added_flores / (added_arandanos + 1) # +1 evita div por cero
        
        print(f"   - Ganancia (x{AUGMENT_FACTOR} copias):")
        print(f"       Epochs verán: +{added_flores} Flores (Bien) | +{added_arandanos} Arándanos (Ruido)")
        print(f"       Ratio de Aumento: Por cada 1 Arándano basura, ganas {ratio_impact:.2f} Flores.")
        
        print(f"   - Balance Final Estimado: {final_flores} Flores vs {final_arandanos} Arándanos")

    print("\n" + "="*60)
    
    # RECOMENDACIÓN AUTOMÁTICA
    # Buscamos el umbral que nos de al menos 1000 flores extra minimizando arándanos
    best_thresh = 0.0
    for thresh in thresholds:
        cands = [img for img in flower_images_data if img['ratio'] >= thresh]
        added_f = sum(img['flores'] for img in cands) * AUGMENT_FACTOR
        if added_f > 1000: # Queremos impacto real
            best_thresh = thresh
            break # Nos quedamos con el más alto que cumpla
            
    if best_thresh > 0:
        print(f"RECOMENDACIÓN: Usa el umbral del {int(best_thresh*100)}%.")
    else:
        print(f"RECOMENDACIÓN: Tus flores siempre están muy rodeadas de arándanos. \n   Considera aumentar TODAS las fotos con flores, o aceptar el 30%.")

# Ejecutar
analyze_flower_density()

🧐 Escaneando 400 archivos...


100%|██████████| 400/400 [00:00<00:00, 2408.74it/s]


ANÁLISIS DE DENSIDAD DE FLORES
   (Global actual: 96203 Flores vs 13313 Arándanos)

🔹 ESCENARIO: Solo aumentar imágenes con > 80% de Flores
   - Imágenes que cumplen: 258 (de 387 con flores)
   - Ganancia (x4 copias):
       Epochs verán: +343632 Flores (Bien) | +24068 Arándanos (Ruido)
       Ratio de Aumento: Por cada 1 Arándano basura, ganas 14.28 Flores.
   - Balance Final Estimado: 439835 Flores vs 37381 Arándanos

🔹 ESCENARIO: Solo aumentar imágenes con > 50% de Flores
   - Imágenes que cumplen: 335 (de 387 con flores)
   - Ganancia (x4 copias):
       Epochs verán: +378120 Flores (Bien) | +38996 Arándanos (Ruido)
       Ratio de Aumento: Por cada 1 Arándano basura, ganas 9.70 Flores.
   - Balance Final Estimado: 474323 Flores vs 52309 Arándanos

🔹 ESCENARIO: Solo aumentar imágenes con > 30% de Flores
   - Imágenes que cumplen: 361 (de 387 con flores)
   - Ganancia (x4 copias):
       Epochs verán: +383148 Flores (Bien) | +46600 Arándanos (Ruido)
       Ratio de Aumento: Por cad